In [1]:
import pandas as pd
import os

base = r"C:\Users\lenovo\Desktop\CV\vanguard_data"

# Load client dataset
df_client = pd.read_csv(os.path.join(base, "df_final_demo.csv.txt"))
print("Client shape:", df_client.shape)

# Load experiment dataset
df_experiment = pd.read_csv(os.path.join(base, "df_final_experiment_clients.csv.txt"))
print("Experiment shape:", df_experiment.shape)

# Load web data part 1
df_web1 = pd.read_csv(os.path.join(base, "df_final_web_data_pt_1.csv.txt"))
print("Web pt1 shape:", df_web1.shape)

# Load web data part 2
df_web2 = pd.read_csv(os.path.join(base, "df_final_web_data_pt_2.csv.txt"))
print("Web pt2 shape:", df_web2.shape)


Client shape: (70609, 9)
Experiment shape: (70609, 2)
Web pt1 shape: (343141, 5)
Web pt2 shape: (412264, 5)


In [2]:
df_web = pd.concat([df_web1, df_web2], ignore_index=True)
print("Combined web shape:", df_web.shape)


Combined web shape: (755405, 5)


In [3]:
import os
import pandas as pd
import numpy as np

# Base folder where all your raw files are stored
base = r"C:\Users\lenovo\Desktop\CV\vanguard_data"

# --- Load client dataset ---
df_client = pd.read_csv(os.path.join(base, "df_final_demo.csv.txt"))
print("Client shape:", df_client.shape)
display(df_client.head())

# --- Load experiment dataset ---
df_experiment = pd.read_csv(os.path.join(base, "df_final_experiment_clients.csv.txt"))
print("Experiment shape:", df_experiment.shape)
display(df_experiment.head())

# --- Load web data part 1 ---
df_web1 = pd.read_csv(os.path.join(base, "df_final_web_data_pt_1.csv.txt"))
print("Web pt1 shape:", df_web1.shape)
display(df_web1.head())

# --- Load web data part 2 ---
df_web2 = pd.read_csv(os.path.join(base, "df_final_web_data_pt_2.csv.txt"))
print("Web pt2 shape:", df_web2.shape)
display(df_web2.head())


Client shape: (70609, 9)


,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0


Experiment shape: (70609, 2)


,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control


Web pt1 shape: (343141, 5)


,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


Web pt2 shape: (412264, 5)


,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,2017-06-06 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,2017-06-01 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,2017-06-01 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,2017-06-01 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,2017-06-01 11:57:58


In [4]:
# Make a copy to avoid touching original
client_clean = df_client.copy()

print("Client shape BEFORE cleaning:", client_clean.shape)

# 1) Drop rows with any null values (≈ 15 rows)
null_rows = client_clean[client_clean.isna().any(axis=1)]
print("Rows with nulls:", len(null_rows))

client_clean = client_clean.dropna().reset_index(drop=True)
print("Client shape AFTER dropping nulls:", client_clean.shape)

# 2) Standardize gendr to M / F / U
def standardize_gender(x):
    if pd.isna(x):
        return "U"
    s = str(x).strip().lower()
    if s in ["m", "male", "man"]:
        return "M"
    if s in ["f", "female", "woman"]:
        return "F"
    return "U"  # anything else → Unknown

if "gendr" in client_clean.columns:
    client_clean["gendr"] = client_clean["gendr"].apply(standardize_gender)
    print("\nGender distribution after cleaning:")
    print(client_clean["gendr"].value_counts(dropna=False))
else:
    print("WARNING: 'gendr' column not found in client data.")

# 3) Convert discrete float columns to Int64
int_cols = ["clnt_tenure_yr", "clnt_tenure_mnth", "num_accts",
            "calls_6_mnth", "logons_6_mnth"]

for col in int_cols:
    if col in client_clean.columns:
        client_clean[col] = client_clean[col].round().astype("Int64")
        print(f"Converted {col} → Int64")

print("\nDtypes after client cleaning:")
print(client_clean.dtypes.head())
display(client_clean.head())


Client shape BEFORE cleaning: (70609, 9)
Rows with nulls: 15
Client shape AFTER dropping nulls: (70594, 9)

Gender distribution after cleaning:
gendr
U    24125
M    23724
F    22745
Name: count, dtype: int64
Converted clnt_tenure_yr → Int64
Converted clnt_tenure_mnth → Int64
Converted num_accts → Int64
Converted calls_6_mnth → Int64
Converted logons_6_mnth → Int64

Dtypes after client cleaning:
client_id             int64
clnt_tenure_yr        Int64
clnt_tenure_mnth      Int64
clnt_age            float64
gendr                object
dtype: object


,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6,73,60.5,U,2,45105.30,6,9
1,2304905,7,94,58.0,U,2,110860.30,6,9
2,1439522,5,64,32.0,U,2,52467.79,6,9
3,1562045,16,198,49.0,M,2,67454.65,3,6
4,5126305,12,145,33.0,F,2,103671.75,0,3


In [5]:
experiment_clean = df_experiment.copy()
print("Experiment shape BEFORE cleaning:", experiment_clean.shape)

# Ensure column name is 'variation'
if "Variation" in experiment_clean.columns and "variation" not in experiment_clean.columns:
    experiment_clean = experiment_clean.rename(columns={"Variation": "variation"})

# Drop rows with null variation (≈ 20,109 rows)
null_var = experiment_clean["variation"].isna().sum()
print("Rows with null 'variation':", null_var)

experiment_clean = experiment_clean[experiment_clean["variation"].notna()].reset_index(drop=True)
print("Experiment shape AFTER dropping null variation:", experiment_clean.shape)

# Standardize variation values to 'control' / 'test'
def normalize_variation(x):
    s = str(x).strip().lower()
    if s.startswith("c"):
        return "control"
    if s.startswith("t"):
        return "test"
    return s  # fallback (shouldn't happen if data is clean)

experiment_clean["variation"] = experiment_clean["variation"].apply(normalize_variation)

print("\nVariation distribution after cleaning:")
print(experiment_clean["variation"].value_counts(dropna=False))
display(experiment_clean.head())


Experiment shape BEFORE cleaning: (70609, 2)
Rows with null 'variation': 20109
Experiment shape AFTER dropping null variation: (50500, 2)

Variation distribution after cleaning:
variation
test       26968
control    23532
Name: count, dtype: int64


,client_id,variation
0,9988021,test
1,8320017,test
2,4033851,control
3,1982004,test
4,9294070,control


In [6]:
# Combine web parts
web_clean = pd.concat([df_web1, df_web2], ignore_index=True)
print("Web combined shape BEFORE cleaning:", web_clean.shape)

# Drop exact duplicate rows (if any)
web_clean = web_clean.drop_duplicates().reset_index(drop=True)
print("Web shape AFTER dropping duplicates:", web_clean.shape)

# Convert date_time to datetime if present
if "date_time" in web_clean.columns:
    web_clean["date_time"] = pd.to_datetime(web_clean["date_time"], errors="coerce")

# Ensure process_step is numeric (Int64)
if "process_step" in web_clean.columns:
    web_clean["process_step"] = pd.to_numeric(web_clean["process_step"], errors="coerce").astype("Int64")

print("\nWeb data info after basic cleaning:")
print(web_clean.info())
display(web_clean.head())


Web combined shape BEFORE cleaning: (755405, 5)
Web shape AFTER dropping duplicates: (744641, 5)

Web data info after basic cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744641 entries, 0 to 744640
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   client_id     744641 non-null  int64         
 1   visitor_id    744641 non-null  object        
 2   visit_id      744641 non-null  object        
 3   process_step  0 non-null       Int64         
 4   date_time     744641 non-null  datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), int64(1), object(2)
memory usage: 29.1+ MB
None


,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,<NA>,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,<NA>,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,<NA>,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,<NA>,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,<NA>,2017-04-17 15:18:04


In [7]:
# Create a local 'data' folder next to the notebook (if not already existing)
output_dir = os.path.join(base, "cleaned_data")
os.makedirs(output_dir, exist_ok=True)

client_path_out = os.path.join(output_dir, "client_clean.csv")
experiment_path_out = os.path.join(output_dir, "experiment_clean.csv")
web_path_out = os.path.join(output_dir, "web_clean.csv")

client_clean.to_csv(client_path_out, index=False)
experiment_clean.to_csv(experiment_path_out, index=False)
web_clean.to_csv(web_path_out, index=False)

print("Saved cleaned files to:")
print(client_path_out)
print(experiment_path_out)
print(web_path_out)


Saved cleaned files to:
C:\Users\lenovo\Desktop\CV\vanguard_data\cleaned_data\client_clean.csv
C:\Users\lenovo\Desktop\CV\vanguard_data\cleaned_data\experiment_clean.csv
C:\Users\lenovo\Desktop\CV\vanguard_data\cleaned_data\web_clean.csv


In [11]:
summary = """
SUMMARY OF DATA CLEANING

1) Client Dataset (df_client / df_final_demo.csv.txt)
- Loaded the raw client dataset.
- Dropped all rows with missing values (around 15 rows).
  These rows are a very small part of the data and removing them is safer than trying to guess values.
- Standardized the 'gendr' column into three categories:
    M = Male
    F = Female
    U = Unknown / Other
- Converted the following columns from float to integer type (Int64) because they represent counts:
    - clnt_tenure_yr
    - clnt_tenure_mnth
    - num_accts
    - calls_6_mnth
    - logons_6_mnth

2) Experiment Dataset (df_experiment / df_final_experiment_clients.csv.txt)
- Loaded the raw experiment roster.
- Renamed the column 'Variation' to 'variation' for consistency.
- Dropped 20,109 rows where 'variation' was null.
  These rows do not belong to either Control or Test group and would bias the A/B test if kept.
- Standardized the 'variation' values to two clean categories:
    - control
    - test

3) Web Dataset (df_web1 + df_web2 / df_final_web_data_pt_1.csv.txt and df_final_web_data_pt_2.csv.txt)
- Loaded both web log files.
- Concatenated the two parts into a single dataframe.
- Removed exact duplicate rows.
- Converted 'date_time' to proper datetime format (when available).
- Converted 'process_step' to a numeric integer type (Int64) so it can be used for step-based analysis.

4) Output
- Saved the cleaned versions of all three datasets into the 'cleaned_data' folder inside:
    C:\\Users\\lenovo\\Desktop\\CV\\vanguard_data\\cleaned_data

These cleaned datasets will be used in the next steps:
- Merging client, experiment, and web data
- Treating outliers in clnt_age, bal, and calls_6_mnth after merging
- Performing EDA, KPI calculation, and hypothesis testing for the Vanguard A/B test.
"""

print(summary)



SUMMARY OF DATA CLEANING

1) Client Dataset (df_client / df_final_demo.csv.txt)
- Loaded the raw client dataset.
- Dropped all rows with missing values (around 15 rows).
  These rows are a very small part of the data and removing them is safer than trying to guess values.
- Standardized the 'gendr' column into three categories:
    M = Male
    F = Female
    U = Unknown / Other
- Converted the following columns from float to integer type (Int64) because they represent counts:
    - clnt_tenure_yr
    - clnt_tenure_mnth
    - num_accts
    - calls_6_mnth
    - logons_6_mnth

2) Experiment Dataset (df_experiment / df_final_experiment_clients.csv.txt)
- Loaded the raw experiment roster.
- Renamed the column 'Variation' to 'variation' for consistency.
- Dropped 20,109 rows where 'variation' was null.
  These rows do not belong to either Control or Test group and would bias the A/B test if kept.
- Standardized the 'variation' values to two clean categories:
    - control
    - test

3) We

In [9]:
import pandas as pd

In [3]:
df_client = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_demo.txt")

In [4]:
df_client.head()

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0


In [5]:
df_client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         70609 non-null  int64  
 1   clnt_tenure_yr    70595 non-null  float64
 2   clnt_tenure_mnth  70595 non-null  float64
 3   clnt_age          70594 non-null  float64
 4   gendr             70595 non-null  object 
 5   num_accts         70595 non-null  float64
 6   bal               70595 non-null  float64
 7   calls_6_mnth      70595 non-null  float64
 8   logons_6_mnth     70595 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 4.8+ MB


In [6]:
df_experiment = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_experiment_clients.txt")

In [7]:
df_experiment.head()

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control


In [17]:
df_experiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  70609 non-null  int64 
 1   Variation  50500 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [8]:
df_webdata1 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_1.txt")

In [9]:
df_webdata1.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


In [18]:
df_webdata1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343141 entries, 0 to 343140
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     343141 non-null  int64 
 1   visitor_id    343141 non-null  object
 2   visit_id      343141 non-null  object
 3   process_step  343141 non-null  object
 4   date_time     343141 non-null  object
dtypes: int64(1), object(4)
memory usage: 13.1+ MB


In [10]:
df_webdata2 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_2.txt")

In [11]:
df_webdata2.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,2017-06-06 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,2017-06-01 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,2017-06-01 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,2017-06-01 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,2017-06-01 11:57:58


In [19]:
df_webdata2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412264 entries, 0 to 412263
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     412264 non-null  int64 
 1   visitor_id    412264 non-null  object
 2   visit_id      412264 non-null  object
 3   process_step  412264 non-null  object
 4   date_time     412264 non-null  object
dtypes: int64(1), object(4)
memory usage: 15.7+ MB


In [13]:
import os
import pandas as pd

cwd = os.getcwd()
candidates = [f for f in os.listdir(cwd) if any(k in f.lower() for k in ["demo","web","experiment"])]

print("Working dir:", cwd)
print("Found candidate files:", candidates)

def try_read(path):
    for sep in [",", "\t", "|", ";"]:
        try:
            df = pd.read_csv(path, sep=sep, nrows=5)
            print(f"SUCCESS reading {path} with sep='{sep}' -> shape preview: {df.shape}")
            display(df.head())
            return True
        except Exception as e:
            #print(f"fail sep={sep}: {e}")
            pass
    print(f"Could not read {path} with common separators.")
    return False

for f in candidates:
    print("\n--- trying:", f)
    try_read(os.path.join(cwd, f))


Working dir: C:\Users\lenovo\Downloads
Found candidate files: ['2.6_web_scraping.ipynb', 'df_final_experiment_clients.txt', 'mysql-installer-web-community-8.0.43.0 (1).msi', 'mysql-installer-web-community-8.0.43.0.msi']

--- trying: 2.6_web_scraping.ipynb
SUCCESS reading C:\Users\lenovo\Downloads\2.6_web_scraping.ipynb with sep='	' -> shape preview: (5, 1)


,{
0,"""cells"": ["
1,{
2,"""cell_type"": ""markdown"","
3,"""id"": ""5171aeed-67b1-4643-bd50-1ffcdfdc9c41"","
4,"""metadata"": {"



--- trying: df_final_experiment_clients.txt
SUCCESS reading C:\Users\lenovo\Downloads\df_final_experiment_clients.txt with sep=',' -> shape preview: (5, 2)


,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control



--- trying: mysql-installer-web-community-8.0.43.0 (1).msi
Could not read C:\Users\lenovo\Downloads\mysql-installer-web-community-8.0.43.0 (1).msi with common separators.

--- trying: mysql-installer-web-community-8.0.43.0.msi
Could not read C:\Users\lenovo\Downloads\mysql-installer-web-community-8.0.43.0.msi with common separators.


In [14]:
# Load & clean df_final_experiment_clients.txt (found in Downloads)
import os
import pandas as pd

# paths
cwd = os.getcwd()
exp_path = os.path.join(cwd, "df_final_experiment_clients.txt")

# read (we already detected comma separation; safe to use read_csv)
df_experiment = pd.read_csv(exp_path)

print("Raw df_experiment shape:", df_experiment.shape)
display(df_experiment.head())

# normalize column name if necessary
if "Variation" in df_experiment.columns and "variation" not in df_experiment.columns:
    df_experiment = df_experiment.rename(columns={"Variation": "variation"})

# count nulls and drop them
null_variation = df_experiment["variation"].isna().sum()
print("Rows with null 'variation':", null_variation)

df_experiment_clean = df_experiment[df_experiment["variation"].notna()].copy().reset_index(drop=True)
print("After dropping nulls shape:", df_experiment_clean.shape)

# normalize text to 'control' / 'test'
df_experiment_clean["variation"] = (
    df_experiment_clean["variation"]
    .astype(str)
    .str.strip()
    .str.lower()
    .map(lambda s: "control" if s.startswith("c") else ("test" if s.startswith("t") else s))
)

print("\nVariation distribution:")
print(df_experiment_clean["variation"].value_counts(dropna=False))

# create data/ if missing and save cleaned experiment file
os.makedirs("data", exist_ok=True)
out_path = os.path.join("data", "df_final_experiment_clients.csv")
df_experiment_clean.to_csv(out_path, index=False)
print(f"\nSaved cleaned experiment roster to: {out_path}")


Raw df_experiment shape: (70609, 2)


,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control


Rows with null 'variation': 20109
After dropping nulls shape: (50500, 2)

Variation distribution:
variation
test       26968
control    23532
Name: count, dtype: int64

Saved cleaned experiment roster to: data\df_final_experiment_clients.csv
